In [1]:
# https://github.com/speechbrain/speechbrain
!pip install speechbrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.8/553.8 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 10.1 MB/s eta 0:00:00


In [2]:
import torch
import torchaudio
from speechbrain.pretrained import SpectralMaskEnhancement

enhance_model = SpectralMaskEnhancement.from_hparams(
    source="speechbrain/metricgan-plus-voicebank",
    savedir="pretrained_models/metricgan-plus-voicebank",
)

# Load and add fake batch dimension
noisy = enhance_model.load_audio(
    "speechbrain/metricgan-plus-voicebank/example.wav"
).unsqueeze(0)

# Add relative length tensor
enhanced = enhance_model.enhance_batch(noisy, lengths=torch.tensor([1.]))

# Saving enhanced signal on disk
torchaudio.save('enhanced.wav', enhanced.cpu(), 16000)


In [3]:
import soundfile
from IPython.display import display, Audio

file_name = 'audio_cache/example.wav'
speech, rate = soundfile.read(file_name)
file_name1 = 'enhanced.wav'
speech1, rate1 = soundfile.read(file_name1)

display(Audio(speech, rate=rate))
display(Audio(speech1, rate=rate1))

## Upload own recordings

In [4]:
from google.colab import files
from IPython.display import display, Audio
import soundfile

uploaded = files.upload()

file_keys = list(uploaded.keys())
file_keys

Saving 332-128985-0073.wav to 332-128985-0073.wav


['332-128985-0073.wav']

In [5]:
!pip install pydub

In [6]:
from pydub import AudioSegment

def resample_audio(audio_path, target_sr=16000):
  audio = AudioSegment.from_file(audio_path)
  resampled_audio = audio.set_frame_rate(target_sr)
  resampled_audio.export(audio_path, format='wav')

In [7]:
resample_audio(file_keys[0], target_sr=16000)

In [8]:
import torch
import torchaudio
from speechbrain.pretrained import SpectralMaskEnhancement

enhance_model = SpectralMaskEnhancement.from_hparams(
    source="speechbrain/metricgan-plus-voicebank",
    savedir="pretrained_models/metricgan-plus-voicebank",
)

# Load and add fake batch dimension
noisy_own = enhance_model.load_audio(
    file_keys[0]
).unsqueeze(0)

# Add relative length tensor
enhanced_own = enhance_model.enhance_batch(noisy_own, lengths=torch.tensor([1.]))

# Saving enhanced signal on disk
torchaudio.save(file_keys[0][:-4] + "_denoised.wav", enhanced_own.cpu(), 16000)

In [11]:
wav_own, sr_own = torchaudio.load(file_keys[0])
wav_own_denoised, sr_own_denoised = torchaudio.load(file_keys[0][:-4] + "_denoised.wav")

In [10]:
from IPython.display import display, Audio
display(Audio(file_keys[0]))
display(Audio(file_keys[0][:-4] + "_denoised.wav"))

## Draw Waveform

In [ ]:
'''
import matplotlib.pyplot as plt
import torch

plt.figure(figsize=(10, 6))

data_org = wav_own
data_denoised = wav_own_denoised

samples = torch.linspace(0, len(data_org) / sr_own, len(data_org))

plt.subplot(2, 1, 1)
plt.title('Original Wavform')
plt.plot(samples, data_org)
plt.xlim(0, len(data_org) / sr_own)

plt.subplot(2, 1, 2)
plt.title('Waveform after enhancement')
plt.plot(samples, data_denoised)
plt.xlim(0, len(data_denoised) / sr_own)
plt.xlabel("Time (s)")
plt.show()
'''